In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from time import sleep

In [2]:
# reading nifty 50 components from wikipedia
tickers = pd.read_html('https://en.wikipedia.org/wiki/NIFTY_50')[1]

In [3]:
tickers_list = tickers['Symbol'].to_list()

In [4]:
tickers_list

['ADANIPORTS',
 'APOLLOHOSP',
 'ASIANPAINT',
 'AXISBANK',
 'BAJAJ-AUTO',
 'BAJFINANCE',
 'BAJAJFINSV',
 'BHARTIARTL',
 'BPCL',
 'BRITANNIA',
 'CIPLA',
 'COALINDIA',
 'DIVISLAB',
 'DRREDDY',
 'EICHERMOT',
 'GRASIM',
 'HCLTECH',
 'HDFC',
 'HDFCBANK',
 'HDFCLIFE',
 'HEROMOTOCO',
 'HINDALCO',
 'HINDUNILVR',
 'ICICIBANK',
 'INDUSINDBK',
 'INFY',
 'ITC',
 'JSWSTEEL',
 'KOTAKBANK',
 'LT',
 'M&M',
 'MARUTI',
 'NESTLEIND',
 'NTPC',
 'ONGC',
 'POWERGRID',
 'RELIANCE',
 'SBIN',
 'SBILIFE',
 'SHREECEM',
 'SUNPHARMA',
 'TATAMOTORS',
 'TATASTEEL',
 'TCS',
 'TATACONSUM',
 'TECHM',
 'TITAN',
 'ULTRACEMCO',
 'UPL',
 'WIPRO']

In [5]:
# fetching ROCE data for nifty 50 stocks from https://www.screener.in/
def fetch_ROCE(stock):  
    try:
        sleep(0.5)
        if stock == 'SBILIFE' or stock == 'NESTLEIND':
            url = f'https://www.screener.in/company/{stock}/'
        else:
            url = f'https://www.screener.in/company/{stock}/consolidated/'
            
        response = requests.get(url)
        html_data = response.text
        soup = BeautifulSoup(html_data, 'html.parser')
        top_ratios = soup.find(id="top-ratios")
        li_elements = top_ratios.find_all('li')
        
        for el in li_elements:
            el_name = el.find(class_="name")
            if el_name.text.strip() == 'ROCE':
                roce_element = el
                break
        if roce_element: 
            return float(roce_element.find(class_="number").text.strip())
    except Exception as e:
        print(e)   

In [6]:
# creating a Data Frame to store ROCE of nifty 50 stocks
nifty_50_roce = pd.DataFrame(index=[0])

In [7]:
# appending .NS to ticker name as yfinance requires .NS while fetching the data
for ticker in tickers_list:
    nifty_50_roce[ticker+'.NS'] = fetch_ROCE(ticker)

In [8]:
nifty_50_roce = nifty_50_roce.T
nifty_50_roce.columns = ['ROCE']

In [9]:
nifty_50_roce.head()

,ROCE
ADANIPORTS.NS,11.20
APOLLOHOSP.NS,17.80
ASIANPAINT.NS,29.70
AXISBANK.NS,5.15
BAJAJ-AUTO.NS,23.40


In [10]:
# fetching ROE data for nifty 50 stocks from https://www.screener.in/
def fetch_ROE(stock):  
    try:
        sleep(0.5)
        if stock == 'SBILIFE' or stock == 'NESTLEIND':
            url = f'https://www.screener.in/company/{stock}/'
        else:
            url = f'https://www.screener.in/company/{stock}/consolidated/'
            
        response = requests.get(url)
        html_data = response.text
        soup = BeautifulSoup(html_data, 'html.parser')
        top_ratios = soup.find(id="top-ratios")
        li_elements = top_ratios.find_all('li')
        
        for el in li_elements:
            el_name = el.find(class_="name")
            if el_name.text.strip() == 'ROE':
                roe_element = el
                break
        if roe_element: 
            return float(roe_element.find(class_="number").text.strip())
    except Exception as e:
        print(e)

In [11]:
# creating a Data Frame to store ROE of nifty 50 stocks
nifty_50_roe = pd.DataFrame(index=[0])

In [12]:
# appending .NS to ticker name as yfinance requires .NS while fetching the data
for ticker in tickers_list:
    nifty_50_roe[ticker+'.NS'] = fetch_ROE(ticker)

In [13]:
nifty_50_roe = nifty_50_roe.T
nifty_50_roe.columns = ['ROE']

In [14]:
nifty_50_roe.head()

,ROE
ADANIPORTS.NS,14.7
APOLLOHOSP.NS,16.6
ASIANPAINT.NS,23.2
AXISBANK.NS,12.7
BAJAJ-AUTO.NS,19.0


In [15]:
# concating ROCE & ROE DataFrame
nifty_50_fundamental_data = pd.concat([nifty_50_roce, nifty_50_roe], axis=1)

In [16]:
nifty_50_fundamental_data.head()

,ROCE,ROE
ADANIPORTS.NS,11.20,14.7
APOLLOHOSP.NS,17.80,16.6
ASIANPAINT.NS,29.70,23.2
AXISBANK.NS,5.15,12.7
BAJAJ-AUTO.NS,23.40,19.0


In [17]:
nifty_50_fundamental_data.shape

(50, 2)

In [18]:
# storing the data to a csv file
nifty_50_fundamental_data.to_csv('Nifty_50_fundamental.csv')